# The LLM Pipeline

In [2]:
import chromadb

client = chromadb.PersistentClient(path='../datasets/processed/chroma_db')
print(client.list_collections())

[Collection(name=summed), Collection(name=instruction), Collection(name=ingredient), Collection(name=name)]


## Loading the models and vector stores using Langchain

In [3]:
import torch
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Initialize models
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

model_path = "TheBloke/Llama-2-13B-chat-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map="auto", revision="main"
)

# Initialize Chroma
chroma_store = Chroma(collection_name='summed', persist_directory='./chroma_db_allrecipes', embedding_function=hf)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    # temperature=0.00,
    # top_p=0.95,
    repetition_penalty=1.15,
)
llm = HuggingFacePipeline(pipeline=pipe)

/home/alvin/miniconda3/envs/lc-update/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Relevant documents can be retrieved from the vector store and used as context

In [3]:
sample_recipe = chroma_store.similarity_search("Beef Stroganoff", k=5)
mushroom_recipe = chroma_store.similarity_search('Portobello Mushroom Stroganoff',k=5)[1].page_content
for doc in sample_recipe:
    print(doc)

beef_recipe = sample_recipe[2].page_content

page_content='Recipe Name: Healthier Beef Stroganoff III, \nIngredients: 2 pounds beef chuck roast, 0.5 teaspoon salt, 0.5 teaspoon ground black pepper, 1 tablespoon butter, 0.5 pound white mushrooms, sliced, 4 green onions, sliced (white and green parts), 2 tablespoons butter, divided, 0.33333334326744 cup white wine, 0.25 cup all-purpose flour, 1.25 cups reduced-sodium beef stock, divided, 1 teaspoon prepared mustard, 0.33333334326744 cup light sour cream, salt and ground black pepper to taste, \nCooking Instructions: 1. Remove any fat and gristle from the roast and cut into strips 1/2-inch thick by 2-inches long. Season with 1/2 teaspoon salt and 1/2 teaspoon pepper.\n2. Melt 1 tablespoon butter in a large skillet over medium heat. Add mushrooms and green onions and cook, stirring occasionally, until mushrooms are browned, about 6 minutes. Remove to a bowl and add 1 tablespoon butter to the skillet. Cook and stir one half the beef strips until browned, about 5 minutes, then remove t

In [4]:
print(beef_recipe)

Recipe Name: Stroganoff, 
Ingredients: 1 pound ground beef, 0.5 cup chopped onion, 1 tablespoon all-purpose flour, 0.5 teaspoon salt, 0.25 teaspoon paprika, 1 (10.75 ounce) can condensed cream of mushroom soup, 1 cup sour cream, 8 ounces egg noodles, 
Cooking Instructions: 1. In a large skillet over medium heat, sauté beef and onions for 10 minutes, or until meat is browned and onion is tender.
2. Stir in flour, salt, and paprika. Add condensed soup, mix well, and cook, uncovered, for 20 minutes.
3. Reduce heat to low and add sour cream, stirring well and allowing to heat through. Cover and set this mixture aside.
4. Cook egg noodles according to package directions. Drain. Serve beef mixture over noodles., 


In [5]:
print(mushroom_recipe)

Recipe Name: Portobello Mushroom Stroganoff, 
Ingredients: 3 tablespoons butter, 1 large onion, chopped, 0.75 pound portobello mushrooms, sliced, 1.5 cups vegetable broth, 1.5 cups sour cream, 3 tablespoons all-purpose flour, 0.25 cup chopped fresh parsley, 8 ounces dried egg noodles, 
Cooking Instructions: 1. Bring a large pot of lightly salted water to a boil. Add egg noodles, and cook until al dente, about 7 minutes. Remove from heat, drain, and set aside.
2. At the same time, melt butter in a large heavy skillet over medium heat. Add onion, and cook, stirring until softened. Turn the heat up to medium-high, and add sliced mushrooms. Cook until the mushrooms are limp and browned. Remove to a bowl, and set aside.
3. In the same skillet, stir in vegetable broth, being sure to stir in any browned bits off the bottom of the pan. Bring to a boil, and cook until the mixture has reduced by 1/3. Reduce heat to low, and return the mushrooms and onion to the skillet.
4. Remove the pan from th

In [6]:
irrelevant_recipe = chroma_store.similarity_search("Chocolate Cake", k=1)[0].page_content
print(irrelevant_recipe)

Recipe Name: Too Much Chocolate Cake, 
Ingredients: 1 (18.25 ounce) package devil's food cake mix, 1 (5.9 ounce) package instant chocolate pudding mix, 1 cup sour cream, 1 cup vegetable oil, 0.5 cup warm water, 4 eggs, 2 cups semisweet chocolate chips, 
Cooking Instructions: 1. Preheat the oven to 350 degrees F (175 degrees C). Generously grease a 12-cup Bundt pan.
2. Mix together cake mix, pudding mix, sour cream, oil, water, and eggs in a large bowl until well-blended.
3. Stir in chocolate chips.
4. Pour batter into the prepared Bundt pan.
5. Bake in the preheated oven until top is springy to the touch and a toothpick inserted into center of cake comes out clean, 50 to 55 minutes.
6. Cool cake thoroughly in the pan for at least 1 1/2 hours before inverting it onto a plate. Sprinkle powdered sugar on top if desired.
7. Enjoy!, 


In [7]:
beef_recipe = """Recipe Name: Beef Stroganoff, 
Ingredients: 1 pound ground beef, 0.5 cup chopped onion, 1 tablespoon all-purpose flour, 0.5 teaspoon salt, 0.25 teaspoon paprika, 1 (10.75 ounce) can condensed cream of mushroom soup, 1 cup sour cream, 8 ounces egg noodles, 
Cooking Instructions: 1. In a large skillet over medium heat, sauté beef and onions for 10 minutes, or until meat is browned and onion is tender.
2. Stir in flour, salt, and paprika. Add condensed soup, mix well, and cook, uncovered, for 20 minutes.
3. Reduce heat to low and add sour cream, stirring well and allowing to heat through. Cover and set this mixture aside.
4. Cook egg noodles according to package directions. Drain. Serve beef mixture over noodles., """

In [8]:
mushroom_recipe = """Recipe Name: Portobello Mushroom Stroganoff, 
Ingredients: 0.75 pound portobello mushrooms, 0.5 cup chopped onion, 1 tablespoon all-purpose flour, 0.5 teaspoon salt, 0.25 teaspoon paprika, 1 (10.75 ounce) can condensed cream of mushroom soup, 1 cup sour cream, 8 ounces egg noodles, 
Cooking Instructions: 1. In a large skillet over medium heat, sauté mushroom and onions for 10 minutes, or until meat is browned and onion is tender.
2. Stir in flour, salt, and paprika. Add condensed soup, mix well, and cook, uncovered, for 20 minutes.
3. Reduce heat to low and add sour cream, stirring well and allowing to heat through. Cover and set this mixture aside.
4. Cook egg noodles according to package directions. Drain. Serve mushroom mixture over noodles., """

In [14]:
beef_pot_pie = chroma_store.similarity_search("Beef Pot Pie", k=5)
for doc in beef_pot_pie:
    print(doc)

beef_stew = beef_pot_pie[4].page_content
beef_pot_pie = beef_pot_pie[0].page_content

page_content='Recipe Name: Beef Pot Pie, \nIngredients: 1 pound sirloin steak, cubed, salt and ground black pepper to taste, 1 (14 ounce) can beef broth, 3 large carrots, diced, 3 potatoes, cubed, 1 cup frozen green peas, thawed, 3 tablespoons cornstarch, 0.33333334326744 cup water, 2 (9 inch) refrigerator pie crusts, \nCooking Instructions: 1. Gather all ingredients.\n2. Brown pieces of meat on all sides in a saucepan over medium heat. Pour in some water to almost cover. Bring to a boil and reduce heat. Simmer until meat is tender and falls apart easily; 2 to 3 hours.\n3. Transfer meat to a large mixing bowl. Shred the meat slightly and add salt and pepper to taste.\n4. Meanwhile, in another 2-quart saucepan over medium heat, pour in beef broth and add carrots and potatoes. Cook until almost tender; 15 to 20 minutes.\n5. Preheat the oven to 350 degrees F (175 degrees C).\n6. When carrots and potatoes are done, transfer to the large mixing bowl with the beef; leaving the liquid in the 

In [10]:
chicken_pot_pie = chroma_store.similarity_search("Chicken Pot Pie", k=5)
for doc in chicken_pot_pie:
    print(doc)
chicken_pot_pie = chicken_pot_pie[3].page_content

page_content='Recipe Name: Healthier Chicken Pot Pie IX, \nIngredients: 1 pound skinless, boneless chicken breast halves - cubed, 1.5 cups sliced carrots, 1 cup frozen green peas, 0.5 cup sliced celery, 3 tablespoons butter, 0.33333334326744 cup chopped onion, 0.33333334326744 cup all-purpose flour, 0.5 teaspoon salt, 0.25 teaspoon ground black pepper, 0.25 teaspoon celery seed, 1.75 cups chicken broth, 0.66666668653488 cup low-fat (1%) milk, 0.25 cup chopped Italian parsley, 2 (9 inch) unbaked whole wheat pie crust, \nCooking Instructions: 1. Preheat oven to 425 degrees F (220 degrees C).\n2. Combine chicken, carrots, peas, and celery in a saucepan. Cover with water. Boil until chicken is no longer pink in the middle and vegetables are fork tender, about 15 minutes. Remove from heat, drain, and set aside.\n3. Cook and stir onions in butter in a saucepan over medium heat, until soft and translucent, about 5 minutes. Stir in flour, salt, pepper, and celery seed. Slowly stir in chicken b

In [11]:
print(beef_pot_pie)

Recipe Name: Beef Pot Pie, 
Ingredients: 1 pound sirloin steak, cubed, salt and ground black pepper to taste, 1 (14 ounce) can beef broth, 3 large carrots, diced, 3 potatoes, cubed, 1 cup frozen green peas, thawed, 3 tablespoons cornstarch, 0.33333334326744 cup water, 2 (9 inch) refrigerator pie crusts, 
Cooking Instructions: 1. Gather all ingredients.
2. Brown pieces of meat on all sides in a saucepan over medium heat. Pour in some water to almost cover. Bring to a boil and reduce heat. Simmer until meat is tender and falls apart easily; 2 to 3 hours.
3. Transfer meat to a large mixing bowl. Shred the meat slightly and add salt and pepper to taste.
4. Meanwhile, in another 2-quart saucepan over medium heat, pour in beef broth and add carrots and potatoes. Cook until almost tender; 15 to 20 minutes.
5. Preheat the oven to 350 degrees F (175 degrees C).
6. When carrots and potatoes are done, transfer to the large mixing bowl with the beef; leaving the liquid in the pan. Combine peas wit

In [12]:
print(chicken_pot_pie)

Recipe Name: Chicken Pot Pie I, 
Ingredients: 4 skinless, boneless chicken breast halves, 1 (10.75 ounce) can condensed cream of chicken soup, 2 (15 ounce) cans mixed vegetables, drained, 1 recipe pastry for a 9 inch double crust pie, 
Cooking Instructions: 1. Boil or steam the chicken breasts until done. Dice.
2. Mix together undiluted soup, canned vegetables, and diced chicken. Pour mixture into pie shell, and cover with the second crust. Crimp the edges, and make slits in the top crust.
3. Bake at 375 degrees F (190 degrees C) for 45 minutes, or until crust is golden brown., 


### Few-shot examples

The intermediate thought process is broken down into several steps:
1. Identify the recipe steps that will need to be modified. For each step in order, follow this process:
a. Isolate and identify the portion of the step to rewrite.
b. Determine the new preparation method with respect to the new ingredient(s)
2. After each step is individually modified, present the final rewritten recipe in a similar format to the base recipe. 

In [13]:
import time

In [17]:
template = f"""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

### Instruction:
Q. Answer the following question by reasoning step-by-step. Here is a recipe for beef stroganoff. Rewrite this recipe to make it vegetarian.
{beef_recipe}

A. To make the recipe vegetarian, the ingredients used to prepare the dish must not include meat. This means that the ground beef must be substituted for a vegetarian option, such as portobello mushrooms.
Steps 1 and 4 directly mention ground beef, so these steps will need to be updated to reflect the change in ingredients. Step 1 should say to "sauté mushroom and onions" instead of "sauté beef and onions".
Step 4 should be changed to mention the mushroom mixture instead of the beef mixture. The modified recipe will look like this:
{mushroom_recipe}

### Input:
Q. Answer the following question by reasoning step-by-step. Here is a recipe for chicken pot pie. Rewrite this recipe to replace the chicken with beef.
{chicken_pot_pie}

### Response:
"""
start = time.time()
test_result = llm(template)
end = time.time()
print(f'Elapsed time: {end-start}s')
print(test_result)

Elapsed time: 39.86314535140991s
To replace the chicken with beef in the recipe, we need to modify the ingredients and the instructions accordingly. Since beef is denser than chicken, we may need to adjust the cooking time as well.

First, we need to substitute the chicken breast halves with an equal amount of beef. We can use beef strips or ground beef, depending on our preference. Let's assume we choose to use ground beef.

Next, we need to adjust the cooking time. Beef takes longer to cook than chicken, so we may need to increase the baking time. Let's add an additional 10-15 minutes to the original baking time of 45 minutes.

Finally, we need to update the instructions to reflect the changes in ingredients and cooking time. Instead of boiling or steaming the chicken, we will need to brown the beef in a pan before adding the other ingredients. We will also need to adjust the amount of liquid in the recipe to account for the difference in moisture content between chicken and beef.

H

In [18]:
template = f"""
You are a helpful, respectful and honest cooking assistant. Always answer as helpfully as possible. Your job is to create a new recipe based on the user's instructions. You will 
be given an original recipe and a request from the user to customize the recipe. To help you customize the recipe, several example recipes will be provided based on the user's request.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
### Example Recipe:
{beef_stew}

### Input:
User Request: Replace chicken for ground beef in the following recipe. Please provide your answer in the form of an informative cooking recipe.
Original Recipe: {chicken_pot_pie}

### Response:
Recipe Name:
"""

test_result = llm(template)
print(test_result)

Instant Pot Ground Beef and Vegetable Stew, 
Ingredients: 1 tablespoon butter, 1 pound ground beef, 4 Yukon Gold potatoes, cubed, 1.5 cups mushrooms, halved, 1 onion, cut into 6 wedges, 2 carrots, cut into 1/2-inch thick slices, 2 cloves garlic, minced, 3 cups beef broth, 1 tablespoon Worcestershire sauce, 1 tablespoon tomato paste, 1 teaspoon salt, 0.5 teaspoon ground black pepper, 0.5 teaspoon dried thyme, 
Cooking Instructions: 1. Gather all ingredients.
2. Turn on a multi-functional pressure cooker (such as Instant Pot) and select the Sauté function. Melt butter in the pot. Cook ground beef until browned, breaking it up into small pieces as it cooks.
3. Add potatoes, mushrooms, onion, carrots, and garlic; cover with beef broth. Stir in Worcestershire sauce, tomato paste, salt, pepper, and thyme.
4. Close and lock the lid. Select Meat/Stew function, according to the manufacturer's instructions; set the timer for 35 minutes. Allow 10 to 15 minutes for pressure to build.
5. Release pr